In [ ]:
import tomli
with open("..\\.streamlit\\secrets.toml",'rb') as f:
	config = tomli.load(f)

In [4]:
from sqlalchemy import create_engine,inspect
def connect_to_postgres(username,password,host,database):
	engine = create_engine(f"postgresql://{username}:{password}@{host}:5432/{database}")  
	return engine

def ag_water():
	engine = connect_to_postgres(	
		username='postgres',
		password=config['password'],
		host=config['host'],
		database='postgres',
	)
	return engine
con = ag_water()

In [9]:
inspector = inspect(con)
table_names = [i for i in inspector.get_table_names() if i.find('TID_') != -1]
table_names

['TID_well_names',
 'TID_well_ownership',
 'TID_well_ownership_single',
 'TID_extractions_yearly_AF',
 'TID_extractions_monthly_AF',
 'TID_current_well_ownership',
 'TID_well_depth_to_water_ft',
 'TID_subsidence_points',
 'TID_subsidence_elevations']

In [84]:
import pandas as pd
dfs = {name:pd.read_sql_table(f"{name}",con) for name in table_names}
table_names = [i for i in dfs.keys()]
table_names

['TID_well_names',
 'TID_well_ownership',
 'TID_well_ownership_single',
 'TID_extractions_yearly_AF',
 'TID_extractions_monthly_AF',
 'TID_current_well_ownership',
 'TID_well_depth_to_water_ft',
 'TID_subsidence_points',
 'TID_subsidence_elevations']

In [86]:
import geopandas as gpd
points = dfs['TID_subsidence_points']
points.head(2)

,index,point_id,point_number,northing,easting,latitude,longitude,notes,source
0,0,AG-20,107.0,2.117338e+06,6.200248e+06,36 38 13.287343 N,120 13 54.744731 W,FD_MAG_NAIL,\\ppeng.com\pzdata\clients\Tranquillity ID-107...
1,1,AG-24,116.0,2.129655e+06,6.189336e+06,36 40 13.671861 N,120 16 10.619000 W,FD_MAG_NAIL,\\ppeng.com\pzdata\clients\Tranquillity ID-107...


In [87]:
elevations = dfs['TID_subsidence_elevations']
elevations.head(2)

,index,point_id,date,elevation,source
0,0,AG-20,2022-07-21,161.30,\\ppeng.com\pzdata\clients\Tranquillity ID-107...
1,1,AG-24,2022-07-21,157.04,\\ppeng.com\pzdata\clients\Tranquillity ID-107...


In [91]:
df = pd.merge(
	points,
	elevations,
	left_on="point_id",
	right_on="point_id",
	how='left',
	suffixes=['_point',"_elevation"],
)
df.head(2)

,index_point,point_id,point_number,northing,easting,latitude,longitude,notes,source_point,index_elevation,date,elevation,source_elevation
0,0,AG-20,107.0,2.117338e+06,6.200248e+06,36 38 13.287343 N,120 13 54.744731 W,FD_MAG_NAIL,\\ppeng.com\pzdata\clients\Tranquillity ID-107...,0,2022-07-21,161.3000,\\ppeng.com\pzdata\clients\Tranquillity ID-107...
1,0,AG-20,107.0,2.117338e+06,6.200248e+06,36 38 13.287343 N,120 13 54.744731 W,FD_MAG_NAIL,\\ppeng.com\pzdata\clients\Tranquillity ID-107...,11,2022-01-04,161.3169,\\ppeng.com\pzdata\clients\Tranquillity ID-107...


In [92]:
def to_decimal_degrees(coord):
	C = coord.split(' ')

	decimal_degree = (float(C[0])) + (float(C[1]) / 60) + (float(C[2]) / 60 / 60)
	if C[3] == 'W':
		return decimal_degree * -1
	else:
		return decimal_degree
coord = "120 13 54.744731 W"
to_decimal_degrees(coord)

df = df.assign(
	lat = lambda y: y.latitude.apply(to_decimal_degrees),
	lon = lambda y: y.longitude.apply(to_decimal_degrees),
)
df.head(2)

,index_point,point_id,point_number,northing,easting,latitude,longitude,notes,source_point,index_elevation,date,elevation,source_elevation,lat,lon
0,0,AG-20,107.0,2.117338e+06,6.200248e+06,36 38 13.287343 N,120 13 54.744731 W,FD_MAG_NAIL,\\ppeng.com\pzdata\clients\Tranquillity ID-107...,0,2022-07-21,161.3000,\\ppeng.com\pzdata\clients\Tranquillity ID-107...,36.637024,-120.231874
1,0,AG-20,107.0,2.117338e+06,6.200248e+06,36 38 13.287343 N,120 13 54.744731 W,FD_MAG_NAIL,\\ppeng.com\pzdata\clients\Tranquillity ID-107...,11,2022-01-04,161.3169,\\ppeng.com\pzdata\clients\Tranquillity ID-107...,36.637024,-120.231874


In [93]:
# gdf = gpd.GeoDataFrame(df,geometry=gpd.points_from_xy(df.northing,df.easting,crs="EPSG:32610"))
gdf = gpd.GeoDataFrame(df,geometry=gpd.points_from_xy(
	df.lon,
	df.lat,
	crs="EPSG:4326"
	))


In [95]:

# gdf.explore(
# 	tooltip = ['point_id'],
# 	marker_kwds = dict(radius=10)
# )


In [98]:
gdf.head(2)

,index_point,point_id,point_number,northing,easting,latitude,longitude,notes,source_point,index_elevation,date,elevation,source_elevation,lat,lon,geometry
0,0,AG-20,107.0,2.117338e+06,6.200248e+06,36 38 13.287343 N,120 13 54.744731 W,FD_MAG_NAIL,\\ppeng.com\pzdata\clients\Tranquillity ID-107...,0,2022-07-21,161.3000,\\ppeng.com\pzdata\clients\Tranquillity ID-107...,36.637024,-120.231874,POINT (-120.23187 36.63702)
1,0,AG-20,107.0,2.117338e+06,6.200248e+06,36 38 13.287343 N,120 13 54.744731 W,FD_MAG_NAIL,\\ppeng.com\pzdata\clients\Tranquillity ID-107...,11,2022-01-04,161.3169,\\ppeng.com\pzdata\clients\Tranquillity ID-107...,36.637024,-120.231874,POINT (-120.23187 36.63702)


In [102]:
import leafmap.foliumap as leafmap
M = leafmap.Map()
M.add_gdf(gdf.drop(columns='date'))
M

The folium plotting backend does not support this function.


In [74]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
	go.Scattergeo(
		locationmode = 'USA-states',

		# lon = df['lon'], lat = df['lat'],
		lon = gdf.geometry.x,
		lat = gdf.geometry.y,
		text= gdf['point_id'],
		# hoverinfo="name",
		# hovertemplate=""
	)
)
fig.update_layout(geo_scope='usa')

fig.show()